In [ ]:
import torch
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
torch.manual_seed(42)

In [ ]:
california_housing = datasets.fetch_california_housing()
california_housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [ ]:
num_features = california_housing.data.shape[1]
print(f'Number of features: {num_features}')

Number of features: 8


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    california_housing.data,
    california_housing.target,
    test_size=0.2,
    random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

last_test_data_point = X_test_scaled[-1]
print(f'Last test data point: {last_test_data_point}')
min_value = np.min(last_test_data_point)
max_value = np.max(last_test_data_point)

print(f'Minimum feature value of the last test data point: {min_value}')
print(f'Maximum feature value of the last test data point: {max_value}')

Last test data point: [-0.17259111 -0.92113763 -0.6058703  -0.14589658  0.21507676  0.05466644
 -0.66608108  0.60445493]
Minimum feature value of the last test data point: -0.9211376319711084
Maximum feature value of the last test data point: 0.6044549279675977


In [ ]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

train_data_shape = X_train_tensor.shape
train_label_shape = y_train_tensor.shape

print(f'Shape of train data tensor: {train_data_shape}')
print(f'Shape of train label tensor: {train_label_shape}')

Shape of train data tensor: torch.Size([16512, 8])
Shape of train label tensor: torch.Size([16512, 1])


In [ ]:
import torch.nn as nn
import torch.optim as optim

class RegressionANN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RegressionANN, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.hidden_layer(x)
        return x

model = RegressionANN(input_dim=8, hidden_dim=16, output_dim=1)

initial_bias = model.hidden_layer.bias.data
print(f'Initial bias in the output layer: {initial_bias}')

Initial bias in the output layer: tensor([0.2272])


In [ ]:
loss_function = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
initial_predictions = model(X_train_tensor)
initial_loss = loss_function(initial_predictions, y_train_tensor)
print(f'Initial loss value: {initial_loss.item()}')

Initial loss value: 4.202902793884277


In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = loss_function(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

print(f'Loss value after 100 iterations: {loss.item()}')

Epoch [10/100], Loss: 2.333400011062622
Epoch [20/100], Loss: 1.119511365890503
Epoch [30/100], Loss: 0.8087179660797119
Epoch [40/100], Loss: 0.728689432144165
Epoch [50/100], Loss: 0.6494651436805725
Epoch [60/100], Loss: 0.6001060009002686
Epoch [70/100], Loss: 0.5567218661308289
Epoch [80/100], Loss: 0.5240655541419983
Epoch [90/100], Loss: 0.497019499540329
Epoch [100/100], Loss: 0.4749816060066223
Loss value after 100 iterations: 0.4749816060066223


In [ ]:
class RegressionANN_64(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RegressionANN_64, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.hidden_layer(x)
        return x

model_64 = RegressionANN_64(input_dim=8, hidden_dim=64, output_dim=1)

optimizer_64 = optim.Adam(model_64.parameters(), lr=0.01)
loss_function_64 = nn.MSELoss()

for epoch in range(num_epochs):
    optimizer_64.zero_grad()
    predictions_64 = model_64(X_train_tensor)
    loss_64 = loss_function_64(predictions_64, y_train_tensor)

    loss_64.backward()
    optimizer_64.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss_64.item()}')

print(f'Loss value after 100 iterations with 64 hidden neurons: {loss_64.item()}')

Epoch [10/100], Loss: 1.0591108798980713
Epoch [20/100], Loss: 0.9692997336387634
Epoch [30/100], Loss: 0.6826358437538147
Epoch [40/100], Loss: 0.5752586126327515
Epoch [50/100], Loss: 0.5252891778945923
Epoch [60/100], Loss: 0.48318079113960266
Epoch [70/100], Loss: 0.4564897418022156
Epoch [80/100], Loss: 0.4391115605831146
Epoch [90/100], Loss: 0.4262136220932007
Epoch [100/100], Loss: 0.41575008630752563
Loss value after 100 iterations with 64 hidden neurons: 0.41575008630752563
